# Test de uso intencionado de la Comunicación entre APIS

#### En este test vamos a mockear el proxy y la base de datos a mano porque solo queremos centranos en la comunicacion a los endpoints de las apis, si los test de integración de db y de proxy funcionan, estos van a funcionar invocados desde la api porque simplemente hacen un rerouting

In [26]:
import sys

# importamos el modulo de funciones
sys.path.append("../..")

In [27]:
from backend.api.api import init
from frontend.helpers.api_functions import API_helper

### Test INIT API helper

In [28]:
api_caller = API_helper()

In [29]:
api_caller is not None

True

### Test INIT API 

In [30]:
import threading

class MockProxy:

    def __init__(self) -> None:
        pass

    async def get_data(self, text, args, id):
        print(text, args, id)
        if args == "question":
            datos = {"respuesta": "got_question", "args": text, "id": id}
        if args == "answer":
            datos = {"respuesta": "got_answer", "args": text, "id": id, "array": [1,2]}
        if args == "example":
            datos = {"respuesta": "got_example", "args": text, "id": id}
        return datos
    
    def validar_usuario(self, user, password):
        return {"id": "MockID"}

    def registrar_usuario(self, user, password):
        return {"id": "MockID"}
    
    def recuperar_valores_array(self, id):
        return {"array": [1,2,3]}
    
    def insertar_valor_array(self, id, valor):
        return {"array": [1,2,3]}
    
class MockFTPServer:

    def __init__(self) -> None:
        pass

    def get_data_as_dic(self):
        return {  
            "user": 1,
            "password": 2,
            "address": 3,
            "port": 4,
        }

fapp = init(MockProxy(), MockFTPServer())
t = threading.Thread(target=fapp.run)

In [31]:
t.start()

###

In [32]:
import time
time.sleep(2) # esperamos a que se inicie la API

### Probamos el endpoint /login

In [33]:
id = await api_caller.login("", "")

In [34]:
id == "MockID"

True

### Probamos el endpoint /register

In [35]:
id = await api_caller.register("", "")

In [36]:
id == "MockID"

True

### Probamos el endpoint /get_array

In [37]:
array = await api_caller.get_array("")

In [38]:
array == [1,2,3]

True

### Probamos el endpoint /post_array

In [39]:
array = await api_caller.post_array("", "")

In [40]:
array == [1,2,3]

True

### Probamos endpoint /health

In [41]:
api_caller.start_health_checker(lambda x: x).start()

### Probamos el endpoint /rag

In [42]:
datos = await api_caller.rag_query()

In [43]:
datos is not None

True

In [44]:
datos

(3, 4, 1, 2)

### Probamos el endpoint /peticion con los 3 formatos

In [45]:
response = await api_caller.query_question({"pregunta": "query"})

In [46]:
response

'got_question'

In [47]:
response = await api_caller.query_example_response({"pregunta": "query"})

In [48]:
response

'got_example'

In [49]:
response = await api_caller.query_for_grading({"pregunta": "query"})

In [50]:
response

('got_answer', [1, 2])

## SE PASAN LOS TESTS